# Homework 2: Nannon
### By Ari Porad
### For COSI 101A with Professor Jordan Pollack, April 12th 2021

## Discussion

For this assignment, I built an implementation of the game Nannon and several different algorithms that could play it. Some high-level discussion of my approach follows in this section, then I'll walk through my implementation in more detail.

### Representation

I'm a big believer that if you choose your data structures right, the rest of your program writes itself. (That's a rough quote from someone, but I can't figure out who.) To that end, I spent a substantial amount of time during this project iterating on my data structures. Originally, I stored the board as a list, where each item in the list represented a position on the board. Legal values were `0` (spot is empty), `1` (current player's checker), and `-1` (opponent's checker). Checkers in the home and goal zones were stored separately. That data structure was simple, which was nice. However, it was easy to get into an illegal state (wrong number of checkers), which I didn't like--a good data structure is one that can't represent an illegal state, after all. It was also hard to manipulate, with lots of loops looking for checkers. I ultimately decided to switch to a representation with an (unordered) list of `Checker`s, each of which has a `Player` (black or white) and a `Position`. Initially, I attempted to have each checker's position be relative _to that checker's home_, which meant that the representation was entirely perspective-independent. That ended up being too complicated to manage, so I settled on a perspective-dependent representation where all checker positions are measured relative to one player's (the perspective player's) home. I'm still not satisfied with this representation--it simultaneously feels a little too complicated, while also needing too much perspective swapping--but it's good enough.

The list of checkers (which is always `2 * checkers_per_player` long) is encapsulated by a `Board`, which also tracks the perspective (if the perspective is Black, then lower indexes are closer to Black's home and vis versa) and `GameConfiguration` (the variant of Nannon, such as `{6,3,6}` or `{8,4,6}`). `Board` contains much of the game logic, including calculating open spots, legal `Move`s, and the winner. The `Move` class trackes a possible move, and is responsible for properly executing it and returning the resulting `Board`. Finally, a `Dicestream` class wraps various iterators that can provide a stream of dice rolls.

### Knowledge-Based Player

### Sticking Points

As mentioned above, picking (and iterating on) a set of datastructures was a big sticking point for this assignment. Building a knowledge-based player was also more difficult than anticipated--the strategies that I intuitively expected to work well weren't actually that effective, whereas the very simple score-based player was actually quite good (as, somewhat surprisingly, the simple last-piece-first player). It's also quite possible that I'm just bad at Nannon--as any of my friends or family can attest, I'm not generally not one for playing games like these the old-fashion way. Additionally, I ran into some very very strange situations that I think are bugs in the Python interpreter, which are more thoroughly document in `cli.py`--but basically, `Player.BLACK.long_str` would sometimes return `'White'`, but only in the PyCharm debugger and inconsistently. That was a fun one.

## Running the Code

I hope I've provided enough information in this report that--as requested--you won't need to run the code. If you do, however, there are a couple of ways to do so.

First, much of the code, especially the game foundation, is unit tested through Python's [doctest][]. Tests can be run from the command line:

```bash
# This prints nothing if the tests pass
$ python3 -m doctest src/*.py
```

Additionally, I built a simple CLI for interacting with the game system:

```bash
$ python3 src/cli.py --help
usage: cli.py [-h] [-n ROUNDS] [-b BOARD_SIZE] [-c CHECKERS] [-d DIE_SIZE] [-s SEED]
              {g,game,t,tournament,r,roundrobin} {first,human,knowledge,last,random,score} [{first,human,knowledge,last,random,score} ...]

positional arguments:
  {g,game,t,tournament,r,roundrobin}
                        game mode
  {first,human,knowledge,last,random,score}
                        player algorithms (2 for game our tournament, 2+ for round robin)

optional arguments:
  -h, --help            show this help message and exit
  -n ROUNDS, --rounds ROUNDS
                        number of matches to run
  -b BOARD_SIZE, --board-size BOARD_SIZE
                        board size
  -c CHECKERS, --checkers CHECKERS
                        checkers per player
  -d DIE_SIZE, --die-size DIE_SIZE
                        die size
  -s SEED, --seed SEED  the seed for the random roll generator

$ python3 src/cli.py game random human  # play against a random player
<Output not shown because it requires humna input>

$ python3 src/cli.py tournament random first last score -n 3000  # run a tournament and show the aggregate results
Playing a Nannon{6,3,6} bulk tournament of 3000 games each between random, first, last, score. seed = 1618284126
Played 6 tournaments of 3000 games each. Took 3s.
Loser → | first   | last    | random  | score
-----------------------------------------------
random  | 53.5%   | 45.8%   | -       | 46.8%
first   | -       | 44.9%   | 46.5%   | 43.0%
last    | 55.1%   | -       | 54.2%   | 50.2%
score   | 57.0%   | 49.8%   | 53.2%   | -

$ python3 src/cli.py tournament last score first --seed 12345  # set the seed to any integer for consistent dice (doesn't affect the random player's decisions). We'll use the same seed as above
Playing a Nannon{6,3,6} bulk tournament of 100 games each between last, score, first. seed = 12345
Played 3 tournaments of 100 games each. Took 0s.
Loser → | first   | last    | score
-------------------------------------
last    | 55.0%   | -       | 48.0%
score   | 52.0%   | 52.0%   | -
first   | -       | 45.0%   | 48.0%
```

[doctest]: https://docs.python.org/3/library/doctest.html

## Results

First off, let's run a round-robin tournament and look at the results. We'll run a lot of matches to get more consistent results.

In [5]:
from bulk_tournament import bulk_tournament
from structs import *
from players import *

results = bulk_tournament([LastPlayerAlgorithm(), FirstPlayerAlgorithm(), ScorePlayerAlgorithm(), RandomPlayerAlgorithm(), KnowledgePlayerAlgorithm()], rounds=10000, config=GameConfiguration(6, 3, 6))

Playing a Nannon{6,3,6} bulk tournament of 10000 games each between last, first, score, random, knowledge. seed = 1618284674
Played 10 tournaments of 10000 games each. Took 12s.                                                                              
Loser →   | first     | knowledge | last      | random    | score    
---------------------------------------------------------------------
last      | 58.0%     | 52.4%     | -         | 52.7%     | 50.9%    
first     | -         | 44.3%     | 42.0%     | 47.2%     | 43.5%    
score     | 56.5%     | 51.9%     | 49.1%     | 53.6%     | -        
random    | 52.8%     | 47.4%     | 47.3%     | -         | 46.4%    
knowledge | 55.7%     | -         | 47.6%     | 52.6%     | 48.1%    


That ran each a 10,000 game match between each of 5 players (for total of 100,000 matches), showing that (for example), the `knowledge` algorithm triumphs over the `andom`